<a href="https://colab.research.google.com/github/namwootree/Portfolio/blob/main/Alphaco_(Deep_Learning_Boot_Camp)/Long-Term%20Program/Text_Summarization/%5BEDA_%EA%B8%B0%EB%B2%95%5DData_Augmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setting

In [ ]:
!pip install transformers
!pip install sentencepiece
!pip install nltk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.2 MB 11.8 MB/s 
     |████████████████████████████████| 596 kB 45.4 MB/s 
     |████████████████████████████████| 86 kB 5.0 MB/s 
     |████████████████████████████████| 6.6 MB 49.8 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.2 MB 12.4 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd

import transformers
from transformers import pipeline
import random

import nltk
from nltk.tokenize import sent_tokenize

from tqdm.notebook import tqdm

In [ ]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
unmasker = pipeline('fill-mask', model='klue/roberta-base')

Downloading:   0%|          | 0.00/546 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/422M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/375 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/243k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/734k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/173 [00:00<?, ?B/s]

In [ ]:
train = pd.read_csv('/content/drive/MyDrive/장기 프로젝트/문서 요약/train_df.csv')

In [ ]:
train = train[['context', 'summary']]
train['context'] = train['context'].str.replace('.', '. ').replace('?', '? ').replace('없습니다.', '없습니다')


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  


In [ ]:
train.shape

(2994, 2)

In [ ]:
train.head()

,context,summary
0,의석을 정돈하여 주시기 바랍니다. 성원이 되었으므로 제207회 완주군의회 임시회 ...,제207회 완주군의회 임시회 제1차 본회의 개의 선포.
1,의사팀장 수고하셨습니다. 먼저 의사일정 제1항 제207회 완주군의회 임시회 회기 ...,제207회 완주군의회 임시회 회기는 8월 26일부터 9월 4일까지 10일간으로 가결됨.
2,다음은 의사일정 제2항 제207회 완주군의회 임시회 회의록 서명의원 선출의 건을 상...,제207회 완주군의회 임시회 회의록 서명의원으로 최등원 의원과 박웅배 의원이 선출됨.
3,다음은 의사일정 제3항 본회의 휴회의 건을 상정합니다. 상임의원회 의정활동을 위하...,8월 27일부터 9월 3일까지 8일간 휴회가 가결됨. 제2차 본회의는 9월 4일 오...
4,의석을 정돈하여 주시기 바랍니다. 성원이 되었으므로 제251회 완주군의회 제1차 ...,제251회 완주군의회 제1차 정례회 제1차 본회의 개의 선포.


# Random Insertion

In [ ]:
def random_insertion(table, unmasker):
  
  dict_random_insertion = {
      'context':[],
      'summary':list(table['summary'])
  }

  for input_text in tqdm(list(table['context'])):

    new_text = ''
    
    sentence_list = sent_tokenize(input_text)

    for sentence in sentence_list:

      orig_text_list = sentence.split()
      len_input = len(orig_text_list)
      
      if len_input == 1:
        new_text += ' ' + sentence
      
      elif len_input <= 3:

        new_text_list = orig_text_list[:1] + ['[MASK]'] + orig_text_list[1:]
        new_mask_sent = ' '.join(new_text_list)

        try:
          augmented_text_list = unmasker(new_mask_sent)
          augmented_text = augmented_text_list[0]['sequence']

          new_augmented_text = ' ' + augmented_text
          new_text += new_augmented_text
          new_text = new_text.lstrip(' ')
        except:
          print(f'오류 : {new_mask_sent}')
          new_text += ' ' + sentence
          pass

      else:
        rand_idx = random.randint(1, len_input-2)

        new_text_list = orig_text_list[:rand_idx] + ['[MASK]'] + orig_text_list[rand_idx:]
        new_mask_sent = ' '.join(new_text_list)

        try:
          augmented_text_list = unmasker(new_mask_sent)
          augmented_text = augmented_text_list[0]['sequence']

          new_augmented_text = ' ' + augmented_text
          new_text += new_augmented_text
          new_text = new_text.lstrip(' ')
        except:
          print(f'오류 : {new_mask_sent}')
          new_text += ' ' + sentence
          pass

    dict_random_insertion['context'].append(new_text+'.')

  df_random_insertion = pd.DataFrame(dict_random_insertion)
  
  return df_random_insertion

In [ ]:
df_random_insertion = random_insertion(train, unmasker)

# Random Replacement

In [ ]:
def random_replacement(table, unmasker):
  
  dict_random_replacement = {
      'context':[],
      'summary':list(table['summary'])
  }

  for input_text in tqdm(list(table['context'])):


    new_text = ''
    
    sentence_list = sent_tokenize(input_text)

    for sentence in sentence_list:

      orig_text_list = sentence.split()
      len_input = len(orig_text_list)
      

      if len_input <= 2:
        new_text += ' ' + sentence

      elif len_input <= 3:
        orig_word = orig_text_list[1]
        new_text_list = orig_text_list.copy()
        new_text_list[1] = '[MASK]'
        new_mask_sent = ' '.join(new_text_list)

        try:
          augmented_text_list = unmasker(new_mask_sent)
          for res in augmented_text_list:
            if res['token_str'] != orig_word:
              augmented_text = res['sequence']
              break

              new_augmented_text = ' ' + augmented_text
              new_text += new_augmented_text
              new_text = new_text.lstrip(' ')

        except:
          print(f'오류 : {new_mask_sent}')
          new_text += ' ' + sentence
          pass

      else:
        rand_idx = random.randint(1, len_input-2)

        orig_word = orig_text_list[rand_idx]
        new_text_list = orig_text_list.copy()
        new_text_list[rand_idx] = '[MASK]'
        new_mask_sent = ' '.join(new_text_list)

        try:
          augmented_text_list = unmasker(new_mask_sent)
          for res in augmented_text_list:
            if res['token_str'] != orig_word:
              augmented_text = res['sequence']
              break
        except:
          print(f'오류 : {new_mask_sent}')
          new_text += ' ' + sentence
          pass

        new_augmented_text = ' ' + augmented_text
        new_text += new_augmented_text
        new_text = new_text.lstrip(' ')

    dict_random_replacement['context'].append(new_text+'.')

  df_random_replacement = pd.DataFrame(dict_random_replacement)

  return df_random_replacement

In [ ]:
df_random_replacement = random_replacement(train, unmasker)

# Random Delection

In [ ]:
def random_delection(table):
  
  dict_random_delection = {
      'context':[],
      'summary':list(table['summary'])
  }

  for input_text in tqdm(list(table['context'])):

    new_text = ''
    
    sentence_list = sent_tokenize(input_text)

    for sentence in sentence_list:

      orig_text_list = sentence.split()
      len_input = len(orig_text_list)
      

      if len_input <= 3:
        new_text += ' ' + sentence

      else:
        rand_idx = random.randint(1, len_input-2)

        orig_word = orig_text_list[rand_idx]
        new_text_list = orig_text_list.copy()
        new_text_list[rand_idx] = ' '
        augmented_text = ' '.join(new_text_list)
        
        new_text_list_2 = augmented_text.split()
        augmented_text = ' '.join(new_text_list_2)

        new_augmented_text = ' ' + augmented_text
        new_text += new_augmented_text
        new_text = new_text.lstrip(' ')

    dict_random_delection['context'].append(new_text+'.')

  df_random_delection = pd.DataFrame(dict_random_delection)

  return df_random_delection

In [ ]:
df_random_replacement = random_delection(train)

  0%|          | 0/2994 [00:00<?, ?it/s]

In [ ]:
df_random_replacement.head()

,context,summary
0,의석을 주시기 바랍니다. 성원이 되었으므로 제207회 완주군의회 임시회 본회의 개의...,제207회 완주군의회 임시회 제1차 본회의 개의 선포.
1,의사팀장 수고하셨습니다. 먼저 의사일정 제1항 완주군의회 임시회 회기 결정의 건을 ...,제207회 완주군의회 임시회 회기는 8월 26일부터 9월 4일까지 10일간으로 가결됨.
2,다음은 의사일정 제2항 제207회 임시회 회의록 서명의원 선출의 건을 상정합니다. ...,제207회 완주군의회 임시회 회의록 서명의원으로 최등원 의원과 박웅배 의원이 선출됨.
3,다음은 의사일정 제3항 본회의 건을 상정합니다. 상임의원회 위하여 8월 27일부터 ...,8월 27일부터 9월 3일까지 8일간 휴회가 가결됨. 제2차 본회의는 9월 4일 오...
4,의석을 정돈하여 바랍니다. 성원이 제251회 완주군의회 제1차 정례회 제1차 본회의...,제251회 완주군의회 제1차 정례회 제1차 본회의 개의 선포.


# Random Swap

In [75]:
def random_swap(table):
  
  dict_random_swap = {
      'context':[],
      'summary':list(table['summary'])
  }

  for input_text in tqdm(list(table['context'])):
    
    sentence_list = sent_tokenize(input_text)

    len_input = len(sentence_list)

    if len_input <= 5:

      rand_idx_1 = random.randint(0, len_input-1)
      rand_idx_2 = random.randint(0, len_input-1)

      sentence_list[rand_idx_1], sentence_list[rand_idx_2] = sentence_list[rand_idx_2], sentence_list[rand_idx_1]

      augmented_text = ' '.join(sentence_list)

      dict_random_swap['context'].append(augmented_text)

    else:

      rand_idx_1 = random.randint(2, len_input-3)
      rand_idx_2 = random.randint(2, len_input-3)

      sentence_list[rand_idx_1], sentence_list[rand_idx_2] = sentence_list[rand_idx_2], sentence_list[rand_idx_1]

      augmented_text = ' '.join(sentence_list)

      dict_random_swap['context'].append(augmented_text)

  df_random_swap = pd.DataFrame(dict_random_swap)

  return df_random_swap

In [76]:
df_random_swap = random_swap(train)

  0%|          | 0/2994 [00:00<?, ?it/s]

In [82]:
train['context'][2]

'다음은 의사일정 제2항 제207회 완주군의회 임시회 회의록 서명의원 선출의 건을 상정합니다.  회의록 서명의원으로는 최등원 의원님과 박웅배 의원님을 선출하고자 하는데 의원여러분 이의 있습니까?(『없습니다』하는 의원 있음)이의가 없으므로 가결되었음을 선포합니다.  그럼 두 분 의원님께서는 본 회기동안 수고하여 주시기 바랍니다'

In [81]:
df_random_swap['context'][2]

'(『없습니다』하는 의원 있음)이의가 없으므로 가결되었음을 선포합니다. 회의록 서명의원으로는 최등원 의원님과 박웅배 의원님을 선출하고자 하는데 의원여러분 이의 있습니까? 다음은 의사일정 제2항 제207회 완주군의회 임시회 회의록 서명의원 선출의 건을 상정합니다. 그럼 두 분 의원님께서는 본 회기동안 수고하여 주시기 바랍니다'